In [1]:
import os
import cv2
import albumentations as A
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm

/home/quirogaalu/miniconda3/envs/tfg/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
import os
import shutil
import random

# Ruta base donde están las carpetas train, valid y test
base_path = '/home/quirogaalu/TFG/datasets/imagenes_todas_det_1024'

train_dir = os.path.join(base_path, 'train')
valid_dir = os.path.join(base_path, 'valid')
test_dir = os.path.join(base_path, 'test')

# Crear carpetas valid y test si no existen
os.makedirs(valid_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Listar imágenes en train
all_images = [f for f in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, f))]

random.seed(33)

# Mezclar la lista de imágenes
random.shuffle(all_images)

# Calcular tamaños de cada partición
total = len(all_images)
train_count = int(0.6 * total)
valid_count = int(0.2 * total)
test_count = total - train_count - valid_count  # para asegurar que suma total

# Separar las listas según partición
train_images = all_images[:train_count]
valid_images = all_images[train_count:train_count + valid_count]
test_images = all_images[train_count + valid_count:]

# Función para mover archivos
def move_files(file_list, source_dir, dest_dir):
    for filename in file_list:
        shutil.move(os.path.join(source_dir, filename), os.path.join(dest_dir, filename))

# Mover imágenes de valid y test desde train
move_files(valid_images, train_dir, valid_dir)
move_files(test_images, train_dir, test_dir)

print(f'Total imágenes: {total}')
print(f'Train: {len(os.listdir(train_dir))}')
print(f'Valid: {len(os.listdir(valid_dir))}')
print(f'Test: {len(os.listdir(test_dir))}')


Total imágenes: 179
Train: 108
Valid: 35
Test: 37


In [3]:
# Ruta base del dataset organizado
base_dir = '/home/quirogaalu/TFG/datasets/imagenes_todas_5_1024'

# Ruta al Excel con conteos reales
conteos_path = '/home/quirogaalu/TFG/ESTADILLO_CONTEOS_FRUTA_LA_CALVILLA.xlsx'
conteos_df = pd.read_excel(conteos_path)

# CSV de salida para regresión
csv_path = '/home/quirogaalu/TFG/labels_regresion_5_1024.csv'

In [4]:
# Definir pipeline de augmentaciones SOLO para train
transform_train  = A.Compose([
    A.HorizontalFlip(p=0.5),                      # Flip horizontal simula cambio de orientación del árbol
    A.RandomBrightnessContrast(                   # Variaciones de luz solar y sombras
        brightness_limit=0.2,
        contrast_limit=0.2,
        p=0.5
    ),
    A.HueSaturationValue(                         # Variaciones de color por maduración o cámara
        hue_shift_limit=0,
        sat_shift_limit=10,
        val_shift_limit=10,
        p=0.3
    ),
    A.Rotate(limit=10, p=0.4),                    # Rotaciones suaves (no alterar distribución espacial drásticamente)
    A.RandomShadow(p=0.2),                        # Sombra de ramas, sol
    A.Resize(1024, 1024)                            # Estándar para el modelo
])

# Para valid/test solo resize
transform_valtest = A.Compose([
    A.Resize(1024, 1024)
])

In [5]:
regression_data = []

def obtener_conteo_real(imagen_nombre):
    """
    Extrae el número de árbol del nombre de imagen y devuelve el conteo real de naranjas
    """
    # Asumimos que el nombre es 'arbolX_imgY.jpg'
    arbol_str = imagen_nombre.split('_')[0]  # 'arbolX'
    arbol_num = int(arbol_str.replace('arbol', ''))
    fila = conteos_df[conteos_df['ARBOL'] == arbol_num]
    if not fila.empty:
        return int(fila.iloc[0]['NARANJAS CONTADAS'])
    else:
        return None

def procesar_subset(subset_name):
    input_folder = os.path.join(base_dir, subset_name)
    output_folder = os.path.join(base_dir, f"{subset_name}_processed")
    os.makedirs(output_folder, exist_ok=True)

    for filename in tqdm(os.listdir(input_folder), desc=f'Procesando {subset_name}'):
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue
        
        conteo_real = obtener_conteo_real(filename)
        if conteo_real is None:
            print(f"⚠️ No se encontró conteo para la imagen {filename}")
            continue

        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Transformar y guardar imagen procesada
        augmented = transform_valtest(image=image)['image']
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR))

        regression_data.append({'ruta_imagen': save_path, 'conteo_real': conteo_real})
        '''
        # Solo para train, crear 3 aumentos extra
        if subset_name == 'train':
            for i in range(3):
                extra_aug = transform_train(image=image)['image']
                aug_name = f"{os.path.splitext(filename)[0]}_aug{i+1}.jpg"
                aug_path = os.path.join(output_folder, aug_name)
                cv2.imwrite(aug_path, cv2.cvtColor(extra_aug, cv2.COLOR_RGB2BGR))
                regression_data.append({'ruta_imagen': aug_path, 'conteo_real': conteo_real})
        '''


# Procesar cada subset
for subset in ['train', 'valid', 'test']:
    procesar_subset(subset)

# Guardar CSV con rutas y conteos para regresión
df_output = pd.DataFrame(regression_data)
df_output.to_csv(csv_path, index=False)
print(f'\n✅ CSV guardado en: {csv_path}')

Procesando test: 100%|██████████████████████████████████████████████████████████████████| 37/37 [00:06<00:00,  5.37it/s]


✅ CSV guardado en: /home/quirogaalu/TFG/labels_regresion_5_1024.csv
